In [1]:
!pip install kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"roshansalunke","key":"16ea77830d34a254c4d6bb9ed8b6d87d"}'}

In [3]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [4]:
! cp kaggle.json ~/.kaggle

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
! kaggle competitions download -c instacart-market-basket-analysis

instacart-market-basket-analysis.zip: Skipping, found more recently modified local copy (use --force to force download)


In [7]:
! mkdir df

mkdir: cannot create directory ‘df’: File exists


In [8]:
! unzip  instacart-market-basket-analysis.zip -d df

Archive:  instacart-market-basket-analysis.zip
replace df/aisles.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace df/departments.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace df/order_products__prior.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace df/order_products__train.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace df/orders.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace df/products.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace df/sample_submission.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [9]:
! pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
! pip install -q bitsandbytes einops wandb datasets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [10]:
!mkdir products
!mkdir dept

mkdir: cannot create directory ‘products’: File exists
mkdir: cannot create directory ‘dept’: File exists


In [11]:
! unzip df/products.csv.zip -d products

Archive:  df/products.csv.zip
replace products/products.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace products/__MACOSX/._products.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [12]:
! unzip df/departments.csv.zip -d dept

Archive:  df/departments.csv.zip
replace dept/departments.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dept/__MACOSX/._departments.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [13]:
import pandas as pd
df_product = pd.read_csv("/content/products/products.csv")
df_dept = pd.read_csv('/content/dept/departments.csv')

In [14]:
df_product.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [15]:
df_joined = pd.merge(df_product, df_dept, on = ['department_id'])

In [16]:
df_joined['text'] = df_joined.apply(lambda row: row['product_name']+" ->: "+row['department'],axis=1)

In [17]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df_joined, test_size=.2, random_state=42)

In [18]:
from datasets import Dataset, DatasetDict

In [19]:
dataset = DatasetDict({
    'train':Dataset.from_pandas(train_df)
})

In [20]:
dataset

DatasetDict({
    train: Dataset({
        features: ['product_id', 'product_name', 'aisle_id', 'department_id', 'department', 'text', '__index_level_0__'],
        num_rows: 39750
    })
})

In [21]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

In [22]:
model_name = 'hfl/chinese-alpaca-2-7b'

bitsandbits = BitsAndBytesConfig(load_in_4bit=True,
                                 bnb_4bit_quant_type='fp4',
                                bnb_4bit_compute_dtype=torch.float16,)

model = AutoModelForCausalLM.from_pretrained(model_name,
                             quantization_config=bitsandbits,
                             trust_remote_code=True)

model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
pip install sentencepiece

In [24]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

In [25]:
from peft import LoraConfig

In [26]:
lora_config = LoraConfig(
    lora_alpha = 16,
    r = 64,
    lora_dropout = .1,
    bias="none",
    task_type="CAUSAL_LM"
)

In [27]:
from transformers import TrainingArguments

In [28]:

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 1
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 120 #500
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [29]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    # train_dataset=data['train'],
    peft_config=lora_config,
    dataset_text_field="text",
    # dataset_text_field="prediction",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


Map:   0%|          | 0/39750 [00:00<?, ? examples/s]

In [30]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [31]:
trainer.train()

wandb: Currently logged in as: roshansalunke81. Use `wandb login --relogin` to force relogin


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,4.380500
2,4.498500
3,4.685300
4,4.552700
5,4.617900
6,4.419000
7,4.627300
8,4.055300
9,4.232400
10,4.122700


TrainOutput(global_step=120, training_loss=2.607952552040418, metrics={'train_runtime': 400.5034, 'train_samples_per_second': 4.794, 'train_steps_per_second': 0.3, 'total_flos': 1241799890927616.0, 'train_loss': 2.607952552040418, 'epoch': 0.05})

In [32]:
lst_test_data = list(test_df['text'])

In [33]:
sample_size = 10
lst_test_data_short = lst_test_data[:sample_size]

In [34]:
import transformers

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # torch_dtype=torch.bfloat16,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
)

sequences = pipeline(
    lst_test_data_short,
    max_length=100,  #200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

for ix,seq in enumerate(sequences):
    print(ix,seq[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


KeyboardInterrupt: ignored

In [35]:
def correct_answer(ans):
  return (ans.split("->:")[1]).strip()

answers = []
for ix,seq in enumerate(sequences):
    # print(ix,seq[0]['generated_text'])
    answers.append(correct_answer(seq[0]['generated_text']))

answers

NameError: ignored

In [ ]:
df_evaluate = test_df.iloc[:sample_size][['product_name','department']]

df_evaluate = df_evaluate.reset_index(drop=True)

df_evaluate['department_predicted'] = answers

df_evaluate